In [1]:
import pandas as pd
import numpy as np
import pygraphviz as pgv

In [12]:
frame = pd.read_csv('/Users/Lutz/Desktop/spiced_projects/discrete-dill-student-code/J3L/notebooks/lutz/supermarket.csv')

In [14]:
frame = df.set_index('timestamp')
frame.index = pd.to_datetime(frame.index)
frame.head()

,customer_no,location,day
timestamp,,,
2019-09-02 07:03:00,2,dairy,Monday
2019-09-02 07:03:00,1,dairy,Monday
2019-09-02 07:04:00,3,dairy,Monday
2019-09-02 07:04:00,8,fruit,Monday
2019-09-02 07:04:00,4,dairy,Monday


In [51]:
frame_group = frame.groupby(['customer_no','day'])['location'].resample('1T').ffill()

In [64]:
frame_group.head(10)

customer_no  day       timestamp          
1            Friday    2019-09-06 07:00:00       dairy
                       2019-09-06 07:01:00       dairy
                       2019-09-06 07:02:00       dairy
                       2019-09-06 07:03:00       dairy
                       2019-09-06 07:04:00      spices
                       2019-09-06 07:05:00    checkout
             Monday    2019-09-02 07:03:00       dairy
                       2019-09-02 07:04:00       dairy
                       2019-09-02 07:05:00    checkout
             Thursday  2019-09-05 07:00:00      drinks
Name: location, dtype: object

In [66]:
frame_group.reset_index()

,customer_no,day,timestamp,location
0,1,Friday,2019-09-06 07:00:00,dairy
1,1,Friday,2019-09-06 07:01:00,dairy
2,1,Friday,2019-09-06 07:02:00,dairy
3,1,Friday,2019-09-06 07:03:00,dairy
4,1,Friday,2019-09-06 07:04:00,spices
...,...,...,...,...
53898,1533,Thursday,2019-09-05 21:48:00,dairy
53899,1534,Thursday,2019-09-05 21:48:00,fruit
53900,1534,Thursday,2019-09-05 21:49:00,checkout
53901,1535,Thursday,2019-09-05 21:48:00,spices


In [67]:
frame_trans = df.reset_index().groupby(['customer_no', 'day'])[['timestamp', 'location']].agg(lambda x: list(x))

In [68]:
frame_trans

timestamp  \
customer_no day                                                            
1           Friday     [2019-09-06 07:00:00, 2019-09-06 07:04:00, 201...   
            Monday            [2019-09-02 07:03:00, 2019-09-02 07:05:00]   
            Thursday          [2019-09-05 07:00:00, 2019-09-05 07:02:00]   
            Tuesday    [2019-09-03 07:02:00, 2019-09-03 07:05:00, 201...   
            Wednesday         [2019-09-04 07:00:00, 2019-09-04 07:02:00]   
...                                                                  ...   
1531        Wednesday         [2019-09-04 21:46:00, 2019-09-04 21:49:00]   
1532        Thursday                               [2019-09-05 21:48:00]   
1533        Thursday                               [2019-09-05 21:48:00]   
1534        Thursday          [2019-09-05 21:48:00, 2019-09-05 21:49:00]   
1535        Thursday          [2019-09-05 21:48:00, 2019-09-05 21:49:00]   

                                        location  
customer_no day                                   
1           Friday     [dairy, spices, checkout]  
            Monday             [dairy, checkout]  
            Thursday          [drinks, checkout]  
            Tuesday    [fruit, drinks, checkout]  
            Wednesday          [fruit, checkout]  
...                                          ...  
1531        Wednesday         [drinks, checkout]  
1532        Thursday                     [dairy]  
1533        Thursday                     [dairy]  
1534        Thursday           [fruit, checkout]  
1535        Thursday          [spices, checkout]  

[7445 rows x 2 columns]

In [75]:
# creating matrix with states before and states after
frame_transition=pd.DataFrame(columns=['before','after'])
index = 0
frame_transition

,before,after


In [76]:
for i in frame_trans.iterrows():
    customer_no = i[0][0]
    day = i[0][1]
    timestamp = i[1][0]
    location = i[1][1]
    #print(sections)
    #location.insert(0,'entrance')
    #timestamp.insert(0,timestamp[0])
    #print(sections)
    #print(time_stamp)
    for j in range(len(location)):
        if (j+1)<len(location):
            frame_transition.loc[index]=(location[j],location[j+1])
            index += 1

In [77]:
frame_transition

,before,after
0,dairy,spices
1,spices,checkout
2,dairy,checkout
3,drinks,checkout
4,fruit,drinks
...,...,...
17427,spices,drinks
17428,drinks,checkout
17429,drinks,checkout
17430,fruit,checkout


In [49]:
frame_cross = pd.crosstab(frame_transition['after'], frame_transition['before'], normalize=0)

""


In [ ]:


#creating transition probability matrix
transition_probability_matrix = pd.crosstab(transition_df['before'], transition_df['after'], normalize=0)
transition_probability_matrix.loc['checkout'] = [1,0,0,0,0,0]

# ordering the columns inside
transition_probability_matrix = pd.DataFrame(transition_probability_matrix,columns=['dairy','drinks','fruit','spices','checkout'],index=['entrance','dairy','drinks','fruit','spices','checkout'])
#print(transition_probability_matrix)